## 1. Connecting to mysql and quering data

In [16]:
import mysql.connector
import pandas as pd

# 1. Connect to MySQL
cnx = mysql.connector.connect(
    host="localhost",    # or your server
    user="root",
    password="Password",
    database="BeerWulf"
)

# 2. Create a cursor and run a query
cursor = cnx.cursor()
query = "SELECT * FROM dim_date LIMIT 5;"
cursor.execute(query)
rows = cursor.fetchall()

# 3. Convert to a DataFrame for display
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
df.head()



,date_key,full_date,day_of_week,month,quarter,year
0,1,1992-01-01,4,1,1,1992
1,2,1992-01-02,5,1,1,1992
2,3,1992-01-03,6,1,1,1992
3,4,1992-01-04,7,1,1,1992
4,5,1992-01-05,1,1,1,1992


## 2. Extra point, adding classifications

In [9]:
import mysql.connector
import pandas as pd

# 1. Connect to MySQL
cnx = mysql.connector.connect(
    host="localhost",    # your server
    user="root",
    password="password",
    database="BeerWulf"
)
cursor = cnx.cursor()

# 2. Alter dim_customer to add a column for account balance classification.
#    We use a try/except block in case the column already exists.
alter_dim_customer = """
ALTER TABLE dim_customer
  ADD COLUMN acctbal_classification VARCHAR(10);
"""
try:
    cursor.execute(alter_dim_customer)
    cnx.commit()
    print("Added acctbal_classification column to dim_customer.")
except Exception as e:
    print("dim_customer column may already exist:", e)

# 3. Update dim_customer to classify account balances.
update_dim_customer = """
UPDATE dim_customer
SET acctbal_classification = CASE
  WHEN c_acctbal < 1000 THEN 'LOW'
  WHEN c_acctbal < 5000 THEN 'MEDIUM'
  ELSE 'HIGH'
END;
"""
cursor.execute(update_dim_customer)
cnx.commit()
print("Updated dim_customer with account balance classification.")

# 4. Alter fact_lineitem to add a revenue column.
alter_fact_lineitem = """
ALTER TABLE fact_lineitem
  ADD COLUMN l_revenue DECIMAL(12,2);
"""
try:
    cursor.execute(alter_fact_lineitem)
    cnx.commit()
    print("Added l_revenue column to fact_lineitem.")
except Exception as e:
    print("fact_lineitem column may already exist:", e)

# 5. Update fact_lineitem to calculate revenue per line item.
#    Here, revenue is defined as: l_extendedprice * (1 - l_discount)
update_fact_lineitem = """
UPDATE fact_lineitem
SET l_revenue = l_extendedprice * (1 - l_discount);
"""
cursor.execute(update_fact_lineitem)
cnx.commit()
print("Updated fact_lineitem with calculated revenue.")

# 6. Optional: Retrieve and display some rows from dim_customer and fact_lineitem

# Query a sample from dim_customer
query_dim_customer = "SELECT * FROM dim_customer LIMIT 5;"
cursor.execute(query_dim_customer)
rows = cursor.fetchall()
df_dim_customer = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
print("Sample from dim_customer:")
display(df_dim_customer)

# Query a sample from fact_lineitem
query_fact_lineitem = "SELECT * FROM fact_lineitem LIMIT 5;"
cursor.execute(query_fact_lineitem)
rows = cursor.fetchall()
df_fact_lineitem = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
print("Sample from fact_lineitem:")
display(df_fact_lineitem)

# 7. Clean up
cursor.close()
cnx.close()


Added acctbal_classification column to dim_customer.
Updated dim_customer with account balance classification.
Added l_revenue column to fact_lineitem.
Updated fact_lineitem with calculated revenue.
Sample from dim_customer:


,customer_key,c_custkey,c_name,c_address,c_phone,c_acctbal,c_mktsegment,c_comment,nation_name,region_name,acctbal_classification
0,1,1,Customer#000000001,"IVhzIApeRb ot,c,E",25-989-741-2988,712,BUILDING,"to the even, regular platelets. regular, ironi...",MOROCCO,AFRICA,LOW
1,2,2,Customer#000000002,"XSTf4,NCwDVaWNe6tEgvwfmRchLXak",23-768-687-3665,122,AUTOMOBILE,l accounts. blithely ironic theodolites integr...,JORDAN,MIDDLE EAST,LOW
2,3,3,Customer#000000003,MG9kdTD2WBHm,11-719-748-3364,7498,AUTOMOBILE,"deposits eat slyly ironic, even instructions....",ARGENTINA,AMERICA,HIGH
3,4,4,Customer#000000004,XxVSJsLAGtn,14-128-190-5944,2867,MACHINERY,"requests. final, regular ideas sleep final accou",EGYPT,MIDDLE EAST,MEDIUM
4,5,5,Customer#000000005,KvpyuHCplrB84WgAiGV6sYpZq7Tj,13-750-942-6364,794,HOUSEHOLD,n accounts will have to unwind. foxes cajole a...,CANADA,AMERICA,LOW


Sample from fact_lineitem:


,fact_lineitem_key,customer_key,supplier_key,part_key,order_date_key,ship_date_key,commit_date_key,receipt_date_key,l_quantity,l_extendedprice,l_discount,l_tax,o_orderstatus,o_shippriority,l_returnflag,l_linestatus,l_linenumber,l_revenue
0,1,370,21,1552,1463,1534,1504,1543,17,24710,0,0,O,0,N,O,1,24710.00
1,2,370,60,674,1463,1564,1520,1572,36,56688,0,0,O,0,N,O,2,56688.00
2,3,370,90,637,1463,1490,1526,1492,8,12301,0,0,O,0,N,O,3,12301.00
3,4,370,9,22,1463,1573,1551,1598,28,25817,0,0,O,0,N,O,4,25817.00
4,5,370,47,241,1463,1551,1535,1553,24,27390,0,0,O,0,N,O,5,27390.00


## 3. Azure stack questions

Describe how you can schedule this process to run multiple times per day.

Answer: We can use schedule triggers, use ADF pipelines to run specific intervals. This way you automatically trigger ETL data

What would you do to cater for data arriving in random order?
 - Add a timestamp to ETL to capture last processed record.
 
What about if the data comes from a stream, and arrives at random times?
 - Use Azure stream analytics, to process real time data.
  
Describe how you would deploy your code to production, and allow for future maitenance. 
 - Use Azure DEVOps, build CI/CD pipeline that automatically deploy Azure Data factory pipeline.
 - Integrate monitoring and logging (Azure monitor) to collect logs and metrics for maintenance.
  

## Data warehouse queries (last question)

In [17]:
# Query a: Bottom 3 nations by revenue
query_a = """
SELECT dc.nation_name, SUM(f.l_revenue) AS total_revenue
FROM fact_lineitem f
JOIN dim_customer dc ON f.customer_key = dc.customer_key
GROUP BY dc.nation_name
ORDER BY total_revenue ASC
LIMIT 3;
"""
cursor.execute(query_a)
rows = cursor.fetchall()
df_a = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
print("Bottom 3 Nations by Revenue:")
display(df_a)

Bottom 3 Nations by Revenue:


,nation_name,total_revenue
0,FRANCE,54431694.00
1,CHINA,65868584.00
2,UNITED STATES,65951276.00
